In [2]:
#importing required libraries

import pandas as pd
import sqlite3
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
#pip install meteostat #install meteostat library.
from meteostat import Stations, Daily

## Extracting Forest Fire Data

Extracting data from fires table in FPA_FOD_20210617.sqlite database sourced from U.S. Department of Agriculture.

In [2]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("Datasets/RDS-2013-0009.5_SQLITE/Data/FPA_FOD_20210617.sqlite")

# Defining the required columns and the SQL query for extraction
required_columns = ["SOURCE_REPORTING_UNIT","SOURCE_REPORTING_UNIT_NAME", "FIRE_YEAR", "DISCOVERY_DATE", 
                    "NWCG_GENERAL_CAUSE", "CONT_DATE", "FIRE_SIZE", 
                    "FIRE_SIZE_CLASS", "LATITUDE", "LONGITUDE", "STATE"]
query = f"SELECT {', '.join(required_columns)} FROM Fires"

# Extract the required columns from the "Fires" dataset
df_fires = pd.read_sql(query, con)

# Display the first few rows of the extracted data
print(df_fires.head())

con.close()

  SOURCE_REPORTING_UNIT SOURCE_REPORTING_UNIT_NAME  FIRE_YEAR  DISCOVERY_DATE  \
0                   511     Plumas National Forest       2005   2/2/2005 0:00   
1                   503   Eldorado National Forest       2004  5/12/2004 0:00   
2                   503   Eldorado National Forest       2004  5/31/2004 0:00   
3                   503   Eldorado National Forest       2004  6/28/2004 0:00   
4                   503   Eldorado National Forest       2004  6/28/2004 0:00   

                           NWCG_GENERAL_CAUSE       CONT_DATE  FIRE_SIZE  \
0  Power generation/transmission/distribution   2/2/2005 0:00       0.10   
1                                     Natural  5/12/2004 0:00       0.25   
2                     Debris and open burning  5/31/2004 0:00       0.10   
3                                     Natural   7/3/2004 0:00       0.10   
4                                     Natural   7/3/2004 0:00       0.10   

  FIRE_SIZE_CLASS   LATITUDE   LONGITUDE STATE  
0      

Clearing invalid values in 'CONT_DATE' column.

In [3]:
df_fires['CONT_DATE'] = df_fires['CONT_DATE'].str.split(' ').str[0]
def check_date_format(date_str):
    """Check if the date string matches the format '%m/%d/%Y'."""
    try:
        pd.to_datetime(date_str, format='%m/%d/%Y')
        return True
    except:
        return False

# Use apply to get a boolean Series where True indicates the format is correct
mask = df_fires['CONT_DATE'].apply(check_date_format)

# Display entries that do not match the format
invalid_entries = df_fires[~mask]

In [4]:
# Filter the DataFrame to retain only entries that match the format
df_fires = df_fires[mask]
# Remove null or blank entries from Column1
df_fires['CONT_DATE'].replace('', pd.NA, inplace=True)  # Convert blanks to NA
df_fires.dropna(subset=['CONT_DATE'], inplace=True)

# converting dates to datetime format
df_fires['DISCOVERY_DATE'] = pd.to_datetime(df_fires['DISCOVERY_DATE'])
df_fires['CONT_DATE'] = pd.to_datetime(df_fires['CONT_DATE'], format='%m/%d/%Y')

Taking subset of data from for discovery date from 2008-2018.( interval decided based on the final dataset size.)

In [5]:
# Filter the DataFrame to include only values after January 1, 2008
fire_data = df_fires[(df_fires['DISCOVERY_DATE'] > '2008-01-01')]

In [6]:
len(fire_data) #length of fire data

620521

## Extracting required weather data

Now, we extract weather data from meteostat library. We define the methods to find the closest weather station to the fire and take the weather data for the duration required.

In [7]:
#function to get closest station
def get_closest_station(latitude, longitude, start_date):
    stations = Stations().nearby(latitude, longitude).fetch()
    if isinstance(start_date, str):
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
    # filter the stations that started on or before the start_date
    valid_stations = stations[(stations['daily_start'] <= start_date)]
    # Reset index to include 'id' as a regular column
    valid_stations = valid_stations.reset_index()
    return valid_stations.iloc[0]



#function to get weather data for specified period
def get_weather_data(latitude, longitude, start_date, end_date):
    closest_station = get_closest_station(latitude, longitude, start_date)

    if closest_station is None:
        print("No suitable weather station found.")
        return None

    data = Daily(closest_station.id, start_date, end_date)
    data = data.fetch()
    data = data.reset_index()
    data.rename(columns = {'time':'Date'}, inplace = True)
    data['Station'] =  closest_station['name']
    data['Station_id'] = closest_station['id']
    return data

We group the fire_data based on the 'SOURCE_REPORTING_UNIT', and use it find the closest weather station to each source reporting unit.

In [8]:
# Group by 'SOURCE_REPORTING_UNIT_NAME', take the first latitude and longitude
grouped = fire_data.groupby(['SOURCE_REPORTING_UNIT']).first().reset_index()
grouped

,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,FIRE_YEAR,DISCOVERY_DATE,NWCG_GENERAL_CAUSE,CONT_DATE,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE
0,102,Beaverhead-Deerlodge National Forest,2008,2008-06-02,Natural,2008-06-02,0.10,A,46.229167,-113.286111,MT
1,103,Bitterroot National Forest,2008,2008-06-29,Recreation and ceremony,2008-06-29,0.10,A,46.246389,-113.904167,MT
2,104,Idaho Panhandle National Forest,2008,2008-07-10,Equipment and vehicle use,2008-07-10,0.10,A,48.333611,-116.983889,ID
3,105,Clearwater National Forest,2008,2008-07-02,Natural,2008-07-02,0.10,A,46.345000,-115.607222,ID
4,108,Custer National Forest,2008,2008-05-30,Natural,2008-05-31,1.40,B,45.311389,-106.419444,MT
...,...,...,...,...,...,...,...,...,...,...,...
4326,WYWRA,Wind River Agency,2008,2008-03-12,Debris and open burning,2008-03-12,1.30,B,43.026300,-108.757100,WY
4327,WYWSFD,Wyoming State Forestry,2010,2010-03-29,Smoking,2010-03-29,0.01,A,42.836051,-108.734301,WY
4328,WYWYS,Wyoming State Forestry,2013,2013-07-05,Natural,2013-07-05,18.00,C,43.451570,-106.133910,WY
4329,WYYNP,Yellowstone National Park,2008,2008-06-13,Arson/incendiarism,2008-06-13,0.10,A,44.556100,-110.401700,WY


We define a method to fetch the wetaher data for each group for the years 2008-2018.

In [9]:
counter = [0]  
def fetch_weather_data(group):
    # Increment the counter
    counter[0] += 1
    
    # Print the counter value conditionally, say every 100 iterations
    if counter[0] % 100 == 0:
        print(f"Processed {counter[0]} groups.")
    
    start_date = '2008-01-01'
    end_date = '2018-12-31'

    weather_df = get_weather_data(group['LATITUDE'], group['LONGITUDE'], start_date, end_date)
    weather_df['SOURCE_REPORTING_UNIT'] = group['SOURCE_REPORTING_UNIT']
    
    return weather_df

We concatenate the weather data.

In [10]:
# Use apply on the grouped dataframe
weather_data_list = grouped.apply(fetch_weather_data, axis=1).tolist()

# Concatenate the results to get the final weather dataset
weather_data = pd.concat(weather_data_list, ignore_index=True)
# drop duplicate weather data
weather_data.drop_duplicates(subset=['SOURCE_REPORTING_UNIT', 'Date'], keep=False, inplace=True)



Processed 100 groups.
Processed 200 groups.
Processed 300 groups.
Processed 400 groups.
Processed 500 groups.
Processed 600 groups.
Processed 700 groups.
Processed 800 groups.
Processed 900 groups.
Processed 1000 groups.
Processed 1100 groups.
Processed 1200 groups.
Processed 1300 groups.
Processed 1400 groups.
Processed 1500 groups.
Processed 1600 groups.
Processed 1700 groups.
Processed 1800 groups.
Processed 1900 groups.
Processed 2000 groups.
Processed 2100 groups.
Processed 2200 groups.
Processed 2300 groups.
Processed 2400 groups.
Processed 2500 groups.
Processed 2600 groups.
Processed 2700 groups.
Processed 2800 groups.
Processed 2900 groups.
Processed 3000 groups.
Processed 3100 groups.
Processed 3200 groups.
Processed 3300 groups.
Processed 3400 groups.
Processed 3500 groups.
Processed 3600 groups.
Processed 3700 groups.
Processed 3800 groups.
Processed 3900 groups.
Processed 4000 groups.
Processed 4100 groups.
Processed 4200 groups.
Processed 4300 groups.


In [11]:
# Display the first few rows of the collected weather data
weather_data.head()

,Date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,Station,Station_id,SOURCE_REPORTING_UNIT
0,2008-01-01,-15.8,-22.8,-6.7,0.0,NaN,NaN,2.5,NaN,1043.3,NaN,Butte / Valley Vista Mobile Home Community,72774,102
1,2008-01-02,-11.4,-17.2,-0.6,0.0,NaN,NaN,2.5,NaN,1028.4,NaN,Butte / Valley Vista Mobile Home Community,72774,102
2,2008-01-03,-6.8,-17.2,3.9,0.0,NaN,NaN,9.4,NaN,1016.3,NaN,Butte / Valley Vista Mobile Home Community,72774,102
3,2008-01-04,0.6,-8.9,4.4,0.0,NaN,NaN,17.6,NaN,1001.0,NaN,Butte / Valley Vista Mobile Home Community,72774,102
4,2008-01-05,-0.6,-7.2,2.2,0.3,NaN,NaN,10.8,NaN,994.4,NaN,Butte / Valley Vista Mobile Home Community,72774,102


## Expanding fire dataset to have daily data.

To merge both datasets first we need to convert the fire dataset to a daily dataset based on the fire duration. Also we impute rows with prior days of the fire for the same duration with fire size class as 'N' to denote days with no fire. This is to create a dataset with balanced fire and no fire days.

In [12]:
# Compute the fire duration
fire_data.loc[:, 'DURATION'] = (fire_data['CONT_DATE'] - fire_data['DISCOVERY_DATE']).dt.days + 1

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [19]:
counter = [0] 
# Define a function to expand fire data
def expand_fire_data(row):
    # Increment the counter
    counter[0] += 1
    
    # Print the counter value conditionally, say every 10000 iterations
    if counter[0] % 10000 == 0:
        print(f"Processed {counter[0]} rows.")
    dates = pd.date_range(start=row['DISCOVERY_DATE'] - pd.Timedelta(days=row['DURATION']), 
                          end=row['CONT_DATE'])
    df = pd.DataFrame({'Date': dates, 'FIRE_SIZE_CLASS': row['FIRE_SIZE_CLASS'], 'FIRE_SIZE': row['FIRE_SIZE']})
    
    df.loc[df['Date'] < row['DISCOVERY_DATE'], 'FIRE_SIZE'] = 0
    df.loc[df['Date'] < row['DISCOVERY_DATE'], 'FIRE_SIZE_CLASS'] = 'N'
    
    for col in fire_data.columns:
        if col not in ['DURATION', 'Date', 'FIRE_SIZE', 'FIRE_SIZE_CLASS']:
            df[col] = row[col]
    
    return df

The code to expand the fire dataset to a daily datset will take much time as it has to iterate through more than 600000 rows.

In [ ]:
# Apply the function
extended_fire_dfs = fire_data.apply(expand_fire_data, axis=1)
extended_fire_df = pd.concat(extended_fire_dfs.values).reset_index(drop=True)

Processed 10000 rows.
Processed 20000 rows.
Processed 30000 rows.
Processed 40000 rows.
Processed 50000 rows.
Processed 60000 rows.
Processed 70000 rows.
Processed 80000 rows.
Processed 90000 rows.
Processed 100000 rows.
Processed 110000 rows.
Processed 120000 rows.
Processed 130000 rows.
Processed 140000 rows.
Processed 150000 rows.
Processed 160000 rows.
Processed 170000 rows.
Processed 180000 rows.
Processed 190000 rows.
Processed 200000 rows.
Processed 210000 rows.
Processed 220000 rows.
Processed 230000 rows.
Processed 240000 rows.
Processed 250000 rows.
Processed 260000 rows.
Processed 270000 rows.
Processed 280000 rows.
Processed 290000 rows.
Processed 300000 rows.
Processed 310000 rows.
Processed 320000 rows.
Processed 330000 rows.
Processed 340000 rows.
Processed 350000 rows.
Processed 360000 rows.
Processed 370000 rows.
Processed 380000 rows.
Processed 390000 rows.
Processed 400000 rows.
Processed 410000 rows.
Processed 420000 rows.
Processed 430000 rows.
Processed 440000 row

## Merging fire and weather data

In [13]:
# matching the key column to merge with fire datset
weather_data['SOURCE_REPORTING_UNIT'] = weather_data['SOURCE_REPORTING_UNIT'].astype(str)

In [18]:
extended_fire_df['Date'] = pd.to_datetime(extended_fire_df['Date'])
weather_data['Date'] = pd.to_datetime(weather_data['Date'])
# Left join weather data with daily fire data
final_df = pd.merge(extended_fire_df, weather_data, 
                    on=['Date', 'SOURCE_REPORTING_UNIT'], 
                    how='left')

final_df.head()

,Date,FIRE_SIZE_CLASS,FIRE_SIZE,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,FIRE_YEAR,DISCOVERY_DATE,NWCG_GENERAL_CAUSE,CONT_DATE,LATITUDE,...,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,Station,Station_id
0,2008-04-07,N,0.0,904,Huron-Manistee National Forest,2008,2008-04-08,Missing data/not specified/undetermined,2008-04-08,44.566389,...,19.0,NaN,NaN,NaN,16.0,NaN,NaN,NaN,Oscoda / Whispering Woods Mobile Home Community,KOSC0
1,2008-04-08,A,0.1,904,Huron-Manistee National Forest,2008,2008-04-08,Missing data/not specified/undetermined,2008-04-08,44.566389,...,9.0,NaN,NaN,NaN,11.0,NaN,NaN,NaN,Oscoda / Whispering Woods Mobile Home Community,KOSC0
2,2008-01-05,N,0.0,905,Mark Twain National Forest,2008,2008-01-06,Debris and open burning,2008-01-06,36.823611,...,14.4,1.3,NaN,184.0,19.4,NaN,1017.9,NaN,Poplar Bluff / Green Forest,72330
3,2008-01-06,C,50.0,905,Mark Twain National Forest,2008,2008-01-06,Debris and open burning,2008-01-06,36.823611,...,18.9,0.0,NaN,191.0,25.2,NaN,NaN,NaN,Poplar Bluff / Green Forest,72330
4,2008-01-07,N,0.0,804,Cherokee National Forest,2008,2008-01-08,Arson/incendiarism,2008-01-08,36.412222,...,18.9,0.0,0.0,NaN,0.0,NaN,NaN,NaN,Bristol / Holston,72335


In [19]:
len(final_df)

3149764

In [20]:
#saving the final dataset.
final_df.to_csv('Datasets/final_dataset2008-18.csv', index=False)

## Dataset for predictive modelling

In [4]:
#looking at the null counts.
na_counts = final_df.isna().sum()
na_counts

Date                                0
FIRE_SIZE_CLASS                     0
FIRE_SIZE                           0
SOURCE_REPORTING_UNIT               0
SOURCE_REPORTING_UNIT_NAME          0
FIRE_YEAR                           0
DISCOVERY_DATE                      0
NWCG_GENERAL_CAUSE                  0
CONT_DATE                           0
LATITUDE                            0
LONGITUDE                           0
STATE                               0
tavg                           876809
tmin                           712985
tmax                           712768
prcp                          1456066
snow                          2487382
wdir                          2817275
wspd                           934332
wpgt                          3148004
pres                          1508077
tsun                          3146777
Station                        701280
Station_id                     701280
dtype: int64

based on the above, We remove the rows with missing na value for tmin tmax prep pres wspd, also we remove column tsun wdir wpgt. we also add column month to use as a predictor variable for the model.

In [8]:
final_df = final_df.dropna(subset=['tmin'])
final_df = final_df.dropna(subset=['wspd'])
final_df = final_df.dropna(subset=['tmax'])
final_df = final_df.dropna(subset=['prcp'])
final_df = final_df.dropna(subset=['pres'])

ctr = ['tsun','wpgt','wdir']
final_df = final_df.drop(columns=ctr)

# assuming days with no snow data as days with no snow.
final_df['snow'] = final_df['snow'].fillna(0)

# creating a new 'month' column
final_df['month'] = final_df['Date'].dt.month 

In [9]:
len(final_df)

1229346

In [10]:
#saving dataset for model training
final_df.to_csv('Datasets/model_dataset2008-18.csv', index=False)